In [ ]:
import qiime2
from qiime2.plugins import demux, dada2, deblur, quality_filter, \
                           metadata, feature_table, alignment, \
                           phylogeny, diversity, emperor, feature_classifier, \
                           taxa, composition, picrust2

import time

## FASTQ

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path './manifest.txt' \
  --output-path './paired-end-demux.qza' \
  --input-format PairedEndFastqManifestPhred33V2

## Cutadapt

In [ ]:
start = time.time()

!qiime cutadapt trim-paired \
    --i-demultiplexed-sequences paired-end-demux.qza \
    --p-cores 12 \
    --p-front-f GTGCCAGCMGCCGCGGTAA \
    --p-front-r GGACTACHVGGGTWTCTAAT \
    --p-match-adapter-wildcards \
    --p-discard-untrimmed \
    --o-trimmed-sequences paired-end-demux-trimmed.qza 

print(time.time()-start)

In [ ]:
!qiime demux summarize --i-data paired-end-demux-trimmed.qza --p-n 30000 --o-visualization paired-end-demux-trimmed.qzv

## 3. DADA2

In [ ]:
start = time.time()

!qiime dada2 denoise-paired \
 --i-demultiplexed-seqs paired-end-demux-trimmed.qza \
 --p-trunc-len-f 188 \
 --p-trunc-len-r 218 \
 --p-trim-left-f 0 \
 --p-trim-left-r 0 \
 --p-n-threads 12 \
 --o-table denoised-sequences-table.qza \
 --o-representative-sequences denoised-sequences-repseq.qza \
 --o-denoising-stats denoised-sequences-stats.qza

print(time.time()-start)

In [ ]:
denoised_sequences_table = qiime2.Artifact.load('denoised-sequences-table.qza')
denoised_sequences_table_vis = feature_table.visualizers.summarize(denoised_sequences_table)
denoised_sequences_table_vis.visualization.save('denoised-sequences-table.qzv')

denoised_sequences_repseq = qiime2.Artifact.load('denoised-sequences-repseq.qza')
denoised_sequences_repseq_vis = metadata.visualizers.tabulate(denoised_sequences_repseq.view(qiime2.Metadata))
denoised_sequences_repseq_vis.visualization.save('denoised-sequences-repseq.qzv')

denoised_sequences_stats = qiime2.Artifact.load('denoised-sequences-stats.qza')
denoised_sequences_stats_vis = metadata.visualizers.tabulate(denoised_sequences_stats.view(qiime2.Metadata))
denoised_sequences_stats_vis.visualization.save('denoised-sequences-stats.qzv')

## Feature Classifier

In [ ]:
#! wget -O "./silva-full-length.qza" "https://data.qiime2.org/2021.8/common/silva-138-99-seqs.qza"
#! wget -O "./silva-full-length-taxonomy.qza" "https://data.qiime2.org/2021.8/common/silva-138-99-tax.qza"

In [ ]:
! qiime feature-classifier extract-reads \
  --i-sequences 'silva-full-length.qza' \
  --p-f-primer GTGCCAGCMGCCGCGGTAA \
  --p-r-primer GGACTACHVGGGTWTCTAAT \
  --p-min-length 100 \
  --p-max-length 400 \
  --p-n-jobs 24 \
  --o-reads 'silva-extract-reads.qza'

In [ ]:
! qiime feature-classifier fit-classifier-naive-bayes \
  --i-reference-reads 'silva-extract-reads.qza' \
  --i-reference-taxonomy 'silva-full-length-taxonomy.qza' \
  --o-classifier 'silva-based-own-classifier.qza'

In [ ]:
!qiime feature-classifier classify-sklearn \
 --i-reads denoised-sequences-repseq.qza \
 --i-classifier silva-based-own-classifier.qza \
 --p-n-jobs 24 \
 --o-classification taxonomy-based-own-classifier.qza

In [ ]:
taxonomy_classification = qiime2.Artifact.load('./taxonomy-based-own-classifier.qza')
taxonomy_classification_vis = metadata.visualizers.tabulate(taxonomy_classification.view(qiime2.Metadata))
#taxonomy_classification_vis.visualization

In [ ]:
!qiime taxa collapse \
  --i-table denoised-sequences-table.qza \
  --i-taxonomy taxonomy-based-own-classifier.qza \
  --p-level 2 \
  --o-collapsed-table phylum-table.qza 

!qiime feature-table relative-frequency \
  --i-table phylum-table.qza \
  --o-relative-frequency-table rel-phylum-table.qza

!qiime tools export --input-path './rel-phylum-table.qza' --output-path './'
!biom convert -i feature-table.biom -o rel-phylum-table.tsv --to-tsv

In [ ]:
!qiime taxa collapse \
  --i-table denoised-sequences-table.qza \
  --i-taxonomy taxonomy-based-own-classifier.qza \
  --p-level 5 \
  --o-collapsed-table family-table.qza 

!qiime feature-table relative-frequency \
  --i-table family-table.qza \
  --o-relative-frequency-table rel-family-table.qza

!qiime tools export --input-path './rel-family-table.qza' --output-path './'
!biom convert -i feature-table.biom -o rel-family-table.tsv --to-tsv

In [ ]:
!qiime taxa collapse \
  --i-table denoised-sequences-table.qza \
  --i-taxonomy taxonomy-based-own-classifier.qza \
  --p-level 6 \
  --o-collapsed-table genus-table.qza 

!qiime feature-table relative-frequency \
  --i-table genus-table.qza \
  --o-relative-frequency-table rel-genus-table.qza

!qiime tools export --input-path './rel-genus-table.qza' --output-path './'
!biom convert -i feature-table.biom -o rel-genus-table.tsv --to-tsv

In [ ]:
table = qiime2.Artifact.load('denoised-sequences-table.qza')
sample_metadata = qiime2.Metadata.load('metadata.txt')
taxonomy_classification = qiime2.Artifact.load('./taxonomy-based-own-classifier.qza')

taxa_bar_plot = taxa.visualizers.barplot(table, taxonomy_classification, sample_metadata)

In [ ]:
!qiime taxa barplot \
  --i-table denoised-sequences-table.qza \
  --i-taxonomy taxonomy-based-own-classifier.qza \
  --m-metadata-file metadata.txt \
  --o-visualization taxa-bar-plots.qzv

## Diversity

In [ ]:
representative_sequences = qiime2.Artifact.load('denoised-sequences-repseq.qza')
rooted_tree = phylogeny.pipelines.align_to_tree_mafft_fasttree(representative_sequences)
rooted_tree.rooted_tree.save('rooted-tree.qza')

#### Alpha-rarefaction

In [ ]:
!qiime diversity alpha-rarefaction \
    --i-table denoised-sequences-table.qza \
    --i-phylogeny rooted-tree.qza \
    --p-max-depth 31895 \
    --m-metadata-file metadata.txt \
    --o-visualization alpha-rarefaction.qzv

#### Diversity metrics

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table denoised-sequences-table.qza \
    --i-phylogeny rooted-tree.qza \
    --p-sampling-depth 31895 \
    --m-metadata-file metadata.txt \
    --output-dir core-metrics-results

## q2longitudinal pairwise-differences: alpha diversity metrics
### shannon, observed-features, faith-pd, pielou-evenness

In [ ]:
!qiime longitudinal pairwise-differences \
  --i-table rel-genus-table.qza --m-metadata-file './metadata.txt' \
  --m-metadata-file ./core-metrics-results/shannon_vector.qza \
  --p-metric shannon_entropy --p-state-column timepoint \
  --p-state-1 baseline \
  --p-state-2 18month \
  --p-individual-id-column usubjid \
  --p-group-column group \
  --o-visualization ./longitudinal/pairwise-differences-shannon.qzv

In [ ]:
pairwise_diff_shannon = qiime2.Visualization.load('./longitudinal/pairwise-differences-shannon.qzv')
#pairwise_diff_shannon

In [ ]:
!qiime longitudinal pairwise-differences \
  --i-table rel-genus-table.qza --m-metadata-file './metadata.txt' \
  --m-metadata-file ./core-metrics-results/observed_features_vector.qza \
  --p-metric observed_features --p-state-column timepoint \
  --p-state-1 baseline \
  --p-state-2 18month \
  --p-individual-id-column usubjid \
  --p-group-column group \
  --o-visualization ./longitudinal/pairwise-differences-observed-features.qzv

In [ ]:
pairwise_diff_observed_features = qiime2.Visualization.load('./longitudinal/pairwise-differences-observed-features.qzv')
#pairwise_diff_observed_features

In [ ]:
!qiime longitudinal pairwise-differences \
  --i-table rel-genus-table.qza --m-metadata-file './metadata.txt' \
  --m-metadata-file ./core-metrics-results/faith_pd_vector.qza \
  --p-metric faith_pd --p-state-column timepoint \
  --p-state-1 baseline \
  --p-state-2 18month \
  --p-individual-id-column usubjid \
  --p-group-column group \
  --o-visualization ./longitudinal/pairwise-differences-faith.qzv

In [ ]:
pairwise_diff_faith = qiime2.Visualization.load('./longitudinal/pairwise-differences-faith.qzv')
#pairwise_diff_faith

In [ ]:
!qiime longitudinal pairwise-differences \
  --i-table rel-genus-table.qza --m-metadata-file './metadata.txt' \
  --m-metadata-file ./core-metrics-results/evenness_vector.qza \
  --p-metric pielou_evenness --p-state-column timepoint \
  --p-state-1 baseline \
  --p-state-2 18month \
  --p-individual-id-column usubjid \
  --p-group-column group \
  --o-visualization ./longitudinal/pairwise-differences-evenness.qzv

In [ ]:
pairwise_diff_evenness = qiime2.Visualization.load('./longitudinal/pairwise-differences-evenness.qzv')
#pairwise_diff_evenness

## q2longitudinal pairwise-distances: beta diversity metrics

In [ ]:
!qiime longitudinal pairwise-distances \
  --i-distance-matrix ./core-metrics-results/unweighted_unifrac_distance_matrix.qza \
  --m-metadata-file './metadata.txt' \
  --p-state-column timepoint \
  --p-state-1 baseline \
  --p-state-2 18month \
  --p-individual-id-column usubjid \
  --p-group-column group \
  --o-visualization ./longitudinal/pairwise-distances-unweighted-unifrac.qzv

In [ ]:
pairwise_dist_uu = qiime2.Visualization.load('./longitudinal/pairwise-distances-unweighted-unifrac.qzv')
#pairwise_dist_uu

In [ ]:
!qiime longitudinal pairwise-distances \
  --i-distance-matrix ./core-metrics-results/weighted_unifrac_distance_matrix.qza \
  --m-metadata-file './metadata.txt' \
  --p-state-column timepoint \
  --p-state-1 baseline \
  --p-state-2 18month \
  --p-individual-id-column usubjid \
  --p-group-column group \
  --o-visualization ./longitudinal/pairwise-distances-weighted-unifrac.qzv

In [ ]:
pairwise_dist_wu = qiime2.Visualization.load('./longitudinal/pairwise-distances-weighted-unifrac.qzv')
#pairwise_dist_wu

## q2-PICRUSt2
- Run PICRUSt2 within QIIME 2 environment. 
- https://library.qiime2.org/plugins/q2-picrust2/13/

In [ ]:
!qiime picrust2 full-pipeline \
  --i-table ./denoised-sequences-table.qza \ # FeatureTable[Frequency]
  --i-seq ./denoised-sequences-repseq.qza \ # FeatureData[Sequence]
  --p-threads 12 \
  --o-ko-metagenome ./picrust2/ko-metagenome.qza \
  --o-ec-metagenome ./picrust2/ec-metagenome.qza \
  --o-pathway-abundance ./picrust2/path-abundance.qza 

In [ ]:
!qiime feature-table summarize \
   --i-table ./picrust2/path-abundance.qza \
   --o-visualization ./picrust2/path-abundance.qzv

In [ ]:
pathway = qiime2.Visualization.load('./picrust2/path-abundance.qzv') # min 1914400
#pathway

In [ ]:
!qiime diversity core-metrics \
    --i-table picrust2/path-abundance.qza \
    --p-sampling-depth 1914400 \
    --m-metadata-file metadata.txt \
    --output-dir picrust2/pathabun-core-metrics-results \
    --p-n-jobs 'auto'

In [ ]:
!qiime longitudinal pairwise-distances \
  --i-distance-matrix ./picrust2/pathabun-core-metrics-results/bray_curtis_distance_matrix.qza \
  --m-metadata-file './metadata.txt' \
  --p-state-column timepoint \
  --p-state-1 baseline \
  --p-state-2 18month \
  --p-individual-id-column usubjid \
  --p-group-column group \
  --o-visualization ./picrust2/longitudinal/pairwise-distances-bray-curtis.qzv

In [ ]:
pairwise_dist_bray_curtis = qiime2.Visualization.load('./picrust2/longitudinal/pairwise-distances-bray-curtis.qzv')
#pairwise_dist_bray_curtis

## PICRUSt2 full pipeline
- Prepare input FASTA and BIOM files to run picrust2_pipeline.py

In [ ]:
## FASTA file for picrust2_pipeline.py
!qiime tools export \
   --input-path ./denoised-sequences-repseq.qza \
   --output-path ./picrust2/ 
#This generates 'dna-sequences.fasta' file

In [ ]:
## biom file for picrust2_pipeline.py
!qiime tools export \
   --input-path ./denoised-sequences-table.qza \
   --output-path ./picrust2/ 

!biom convert \
-i ./picrust2/feature-table.biom \
  -o ./picrust2/feature-table.tsv \
  --to-tsv

In [ ]:
## Install PICRUSt2 conda environment and run the below query
# picrust2_pipeline.py -s dna-sequences.fasta -i feature-table.biom -o picrust2_out_pipeline -p 12